# Importing libraries

In [16]:
import os
import torch
import wandb
import evaluate
import numpy as np
from PIL import Image
import io
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    default_data_collator
)
from torchvision import transforms
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, roc_auc_score


# Initializing wandb

In [17]:
wandb.login(key='90c6ded1d5897b142f28dcf8658ff0ead673bf4d')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Nitro\_netrc


True

# Load dataset

In [18]:
dataset = load_dataset("derek-thomas/ScienceQA")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
        num_rows: 12726
    })
    validation: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
        num_rows: 4241
    })
    test: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
        num_rows: 4241
    })
})


# Initializing tokenizer and the model

In [20]:
model_name = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Setting up the image transformations

In [21]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Creating prompt format

In [22]:

def create_prompt(question, choices, hint=None, lecture=None):
    """Create a structured prompt with clear instructions."""
    choices_text = '\n'.join([f"{chr(65 + i)}. {choice}" for i, choice in enumerate(choices)])
    hint_text = f"Hint: {hint}\n" if hint else ""
    lecture_text = f"Lecture Context: {lecture}\n" if lecture else ""

    return (
        "You are a specialized science tutor assisting a student. Your goal is to answer "
        "the question below based on scientific principles, leveraging both the lecture context "
        "and any provided hints.\n\n"
        f"{lecture_text}{hint_text}Question: {question}\n\n"
        f"Choices:\n{choices_text}\n\n"
        "Provide the most accurate answer based on logical reasoning and scientific knowledge:"
    )



# Processing examples

def process_example(example):
    """Process a single example from the dataset"""
    prompt = create_prompt(example["question"], example["choices"])

    encoding = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    pixel_values = None
    if example["image"] is not None:
        try:
            if isinstance(example["image"], dict) and "bytes" in example["image"]:
                image = Image.open(io.BytesIO(example["image"]["bytes"]))
            else:
                image = example["image"]
            pixel_values = image_transform(image)
        except Exception as e:
            print(f"Error processing image: {e}")
            pixel_values = torch.zeros((3, 224, 224))
    else:
        pixel_values = torch.zeros((3, 224, 224))

    return {
        "input_ids": encoding["input_ids"][0],
        "attention_mask": encoding["attention_mask"][0],
        "pixel_values": pixel_values,
        "labels": example["answer"]
    }


# Processing dataset

In [23]:
processed_dataset = dataset.map(
    process_example,
    remove_columns=dataset["train"].column_names
)

Map: 100%|██████████| 4241/4241 [00:34<00:00, 124.15 examples/s]


In [24]:
def process_dataset(dataset):
    """Process an entire dataset."""
    processed_data = [process_example(example) for example in dataset]
    return processed_data


# Evaluation function

In [25]:
metrics = {
    "accuracy": evaluate.load("accuracy"),
    "precision": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1")
}

def compute_metrics(eval_pred):
    """Compute evaluation metrics"""
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)

    results = {
        "accuracy": metrics["accuracy"].compute(predictions=predictions, references=labels)["accuracy"],
        "precision": metrics["precision"].compute(predictions=predictions, references=labels, average='weighted')["precision"],
        "recall": metrics["recall"].compute(predictions=predictions, references=labels, average='weighted')["recall"],
        "f1": metrics["f1"].compute(predictions=predictions, references=labels, average='weighted')["f1"]
    }

    results["confusion_matrix"] = confusion_matrix(labels, predictions).tolist()

    if len(np.unique(labels)) == 2:
        results["roc_auc"] = roc_auc_score(labels, predictions)

    return results



# Setting up training arguments

In [39]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  
    save_total_limit=2,     
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)

# Initializing the model

In [40]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(set(processed_dataset["train"]["labels"]))
)


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Initializing the trainer

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics
)


C:\Users\Nitro\AppData\Local\Temp\ipykernel_53036\3089658747.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Trainning the model

In [42]:
trainer.train()

  1%|▏         | 500/39800 [1:06:06<85:37:53,  7.84s/it]

{'loss': 1.0711, 'grad_norm': 3.2666122913360596, 'learning_rate': 1.9748743718592968e-05, 'epoch': 0.63}


  2%|▏         | 796/39800 [1:45:14<70:48:06,  6.53s/it]c:\Python311\Lib\site-packages\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

  2%|▏         | 796/39800 [1:55:01<70:48:06,  6.53s/it]

{'eval_loss': 0.9749144911766052, 'eval_accuracy': 0.41664701721292147, 'eval_precision': 0.3348470063674198, 'eval_recall': 0.41664701721292147, 'eval_f1': 0.3711568784800813, 'eval_confusion_matrix': [[627, 982, 0, 0, 0], [620, 1140, 0, 0, 0], [593, 7, 0, 0, 0], [265, 0, 0, 0, 0], [7, 0, 0, 0, 0]], 'eval_runtime': 586.5901, 'eval_samples_per_second': 7.23, 'eval_steps_per_second': 0.227, 'epoch': 1.0}


  3%|▎         | 1000/39800 [2:22:10<84:35:58,  7.85s/it]  

...

100%|██████████| 39800/39800 [98:12:11<00:00,  6.93s/it]

{'eval_loss': 1.418050765991211, 'eval_accuracy': 0.801697712803584, 'eval_precision': 0.8008813862004194, 'eval_recall': 0.801697712803584, 'eval_f1': 0.80126711689309, 'eval_confusion_matrix': [[1296, 225, 71, 16, 1], [225, 1447, 69, 18, 1], [75, 57, 443, 25, 0], [17, 17, 17, 214, 0], [3, 2, 0, 2, 0]], 'eval_runtime': 622.639, 'eval_samples_per_second': 6.811, 'eval_steps_per_second': 0.214, 'epoch': 50.0}


100%|██████████| 39800/39800 [98:12:16<00:00,  8.88s/it]

{'train_runtime': 353536.8485, 'train_samples_per_second': 1.8, 'train_steps_per_second': 0.113, 'train_loss': 0.2421450955664093, 'epoch': 50.0}


TrainOutput(global_step=39800, training_loss=0.2421450955664093, metrics={'train_runtime': 353536.8485, 'train_samples_per_second': 1.8, 'train_steps_per_second': 0.113, 'total_flos': 4.87731404788992e+16, 'train_loss': 0.2421450955664093, 'epoch': 50.0})

# Evaluating on the test set

In [43]:
test_results = trainer.evaluate(processed_dataset["test"])
print("Test Results:", test_results)

100%|██████████| 133/133 [10:21<00:00,  4.64s/it]c:\Python311\Lib\site-packages\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 133/133 [10:21<00:00,  4.67s/it]

Test Results: {'eval_loss': 1.1231170892715454, 'eval_accuracy': 0.794388116010375, 'eval_precision': 0.7933377948889511, 'eval_recall': 0.794388116010375, 'eval_f1': 0.7937734178572083, 'eval_confusion_matrix': [[1383, 235, 64, 18, 0], [248, 1338, 78, 25, 0], [65, 49, 450, 19, 0], [14, 30, 19, 198, 0], [0, 7, 1, 0, 0]], 'eval_runtime': 626.1541, 'eval_samples_per_second': 6.773, 'eval_steps_per_second': 0.212, 'epoch': 50.0}
